In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import folium
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
%reload_ext autotime

time: 0 ns (started: 2024-07-28 18:10:58 +09:00)


In [2]:
def get_season(date: pd.Timestamp) -> str:
    if date.month in (3, 4, 5):
        return '봄'
    elif date.month in (6, 7, 8):
        return '여름'
    elif date.month in (9, 10, 11):
        return '가을'
    else:
        return '겨울'

time: 0 ns (started: 2024-07-28 18:10:58 +09:00)


In [3]:
# 0. 데이터 전처리
# 여행객 데이터와 방문지 정보를 불러오기
traveler_master = pd.read_csv('./data/bami/total/여행객 Master.csv')
traveler_master['계절'] = traveler_master['TRAVEL_STATUS_YMD'].apply(lambda x: pd.to_datetime(x.split('~')[0])).apply(get_season)

visit_info = pd.read_csv('./data/bami/total/방문지정보.csv', dtype={'X_COORD': str, 'Y_COORD': str})
visit_info['계절'] = pd.to_datetime(visit_info['VISIT_START_YMD']).apply(get_season)


time: 2.84 s (started: 2024-07-28 18:10:58 +09:00)


In [4]:
visit_info

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,region,계절
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,4.159012e+09,capital,봄
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.0233392,37.2878779,NaN,NaN,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장,60.0,2,N,10.0,NaN,4.0,3.0,4.0,NaN,capital,봄
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127.0251435,37.287791,NaN,NaN,POI010000006N1USC,창룡문,30.0,2,N,1.0,NaN,4.0,4.0,4.0,NaN,capital,봄
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127.0176261,37.2875455,NaN,NaN,POI01000TR021821V,수원화성 화홍문,60.0,2,N,10.0,NaN,4.0,3.0,3.0,NaN,capital,봄
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,390.0,21,NaN,NaN,NaN,NaN,NaN,NaN,4.159012e+09,capital,봄
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79160,2308250002,h_h003280,2,법주사,2023-08-25,2023-08-25,충북 보은군 속리산면 법주사로 405,충북 보은군 속리산면 사내리 209,127.8329575,36.5417208,NaN,NaN,POI01000TR042115V,법주사,90.0,2,N,1.0,NaN,5.0,5.0,5.0,NaN,western,여름
79161,2308250003,h_h003280,3,정이품송,2023-08-25,2023-08-25,NaN,충북 보은군 속리산면 상판리 241,127.8155813,36.5174042,NaN,NaN,POI01000TR020212V,정이품송,30.0,1,N,1.0,NaN,5.0,5.0,5.0,NaN,western,여름
79162,2308250004,h_h003280,5,옛 고을 식당,2023-08-25,2023-08-25,충북 보은군 속리산면 법주사로 250,충북 보은군 속리산면 사내리 280-1,127.8214922,36.5304561,NaN,NaN,POI010000006UDEAX,옛고을식당,30.0,11,N,7.0,NaN,5.0,5.0,5.0,NaN,western,여름
79163,2308250005,h_h003280,4,문의 청남대 휴게소 청주 방향,2023-08-25,2023-08-25,충북 청주시 상당구 문의면 당진영덕고속도로 123,충북 청주시 상당구 문의면 남계리 산 59-6,127.4844718,36.5442568,NaN,NaN,POI01000TR028239V,문의청남대휴게소 청주방향,30.0,4,N,9.0,NaN,5.0,5.0,5.0,NaN,western,여름


time: 31 ms (started: 2024-07-28 18:11:01 +09:00)


1. **위도, 경도** 값이 없는 일부 데이터들의 값들을 채움
2. '올레길', '올레길 주차장' 처럼 **달리 표기된 장소를 하나로** 통일
3. 방문지에서 **사적 장소**(집, 친구/친지집, 사무실), **기타**(편의점, 피시방, 사우나 등)은 데이터에서 배제
   - codeB를 이용해서 完
   - 논의해야 할 점
     - 기타를 포함할지?
     - 역, 터미널, 고속도로 휴게소를 포함할지?(선행 제주도 프로젝트에서는 포함한 것으로 보임)
4. **식당**이면 RES, **관광지**면 TOU로 데이터 구분
   - codeB를 이용해서 code 10(식당)이 아니면 TOU로 넣으면 될 것 같음
   - but, 다른 code Num을 갖는 것들 중 예외 처리 필요 있음(상업지구, 상점, 지역축제 등)

In [5]:
visit_pre = pd.read_csv('./data/bami/total/방문지정보.csv')

# 12: 기타, 21: 집, 22: 친구/친지집, 23: 사무실
del_area_type = {12, 21, 22, 23}

# 사적 장소 제거
visit_pre = visit_pre[~visit_pre['VISIT_AREA_TYPE_CD'].isin(del_area_type)]

visit_pre

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,region
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.023339,37.287878,NaN,NaN,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장,60.0,2,N,10.0,NaN,4.0,3.0,4.0,NaN,capital
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127.025143,37.287791,NaN,NaN,POI010000006N1USC,창룡문,30.0,2,N,1.0,NaN,4.0,4.0,4.0,NaN,capital
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127.017626,37.287546,NaN,NaN,POI01000TR021821V,수원화성 화홍문,60.0,2,N,10.0,NaN,4.0,3.0,3.0,NaN,capital
6,2304300002,e_e000006,2,청량리역 경춘선,2023-04-30,2023-04-30,서울 동대문구 왕산로 214,서울 동대문구 전농동 591-53,127.048636,37.581274,NaN,NaN,POI01000TR031679V,청량리역 경춘선,30.0,9,Y,6.0,NaN,5.0,5.0,5.0,NaN,capital
7,2304300003,e_e000006,3,청평역 경춘선,2023-04-30,2023-04-30,경기 가평군 청평면 청평역로 97-33,경기 가평군 청평면 청평리 174,127.426623,37.735516,NaN,NaN,POI01000000BYKGW1,청평(신)역,0.0,9,N,1.0,NaN,5.0,5.0,5.0,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79157,2307070004,h_h001459,14,경기 광주 휴게소 주유소 원주 방향,2023-07-07,2023-07-07,경기 광주시 곤지암읍 만삼로 270,경기 광주시 곤지암읍 삼합리 265-1,127.437956,37.378886,NaN,NaN,POI01000000BVXA1A,경기광주휴게소주유소 원주방향,30.0,9,N,9.0,NaN,5.0,5.0,5.0,NaN,western
79160,2308250002,h_h003280,2,법주사,2023-08-25,2023-08-25,충북 보은군 속리산면 법주사로 405,충북 보은군 속리산면 사내리 209,127.832958,36.541721,NaN,NaN,POI01000TR042115V,법주사,90.0,2,N,1.0,NaN,5.0,5.0,5.0,NaN,western
79161,2308250003,h_h003280,3,정이품송,2023-08-25,2023-08-25,NaN,충북 보은군 속리산면 상판리 241,127.815581,36.517404,NaN,NaN,POI01000TR020212V,정이품송,30.0,1,N,1.0,NaN,5.0,5.0,5.0,NaN,western
79162,2308250004,h_h003280,5,옛 고을 식당,2023-08-25,2023-08-25,충북 보은군 속리산면 법주사로 250,충북 보은군 속리산면 사내리 280-1,127.821492,36.530456,NaN,NaN,POI010000006UDEAX,옛고을식당,30.0,11,N,7.0,NaN,5.0,5.0,5.0,NaN,western


time: 313 ms (started: 2024-07-28 18:11:01 +09:00)


In [32]:
from fuzzywuzzy import fuzz, process
from tqdm import tqdm

# Define a function to unify place names based on fuzzy matching
def unify_place_names(df, column_name, threshold=95):
    unique_names = df[column_name].unique()
    unified_names = {}

    for name in tqdm(unique_names):
        # Find the best match in the current dictionary of unified names
        match = process.extractOne(name, list(unified_names.keys()), scorer=fuzz.ratio)
        if match:
            best_match, score = match
            if score >= threshold:
                unified_names[name] = unified_names[best_match]
            else:
                unified_names[name] = name
        else:
            unified_names[name] = name

    # Replace the names in the dataframe
    df[f'{column_name}_NP'] = df[column_name].map(unified_names)

    return df

# Apply the function to the VISIT_AREA_NM column
t = unify_place_names(visit_pre, 'VISIT_AREA_NM')

100%|██████████| 29358/29358 [30:17<00:00, 16.15it/s]

time: 30min 17s (started: 2024-07-28 20:49:36 +09:00)


In [33]:
t[t['VISIT_AREA_NM'] != t['VISIT_AREA_NM_NP']][['VISIT_AREA_NM', 'VISIT_AREA_NM_NP']]

,VISIT_AREA_NM,VISIT_AREA_NM_NP
889,소테츠호텔즈 더 스페라 지르 서울 명동,소테츠호텔즈 더 스페라지르 서울 명동
890,소테츠호텔즈 더 스페라 지르 서울 명동,소테츠호텔즈 더 스페라지르 서울 명동
2134,현대 프리미엄 아울렛 스페이스 원,현대 프리미엄 아울렛 스페이스원
2320,본가신숙희진골막국수,본가신숙희 진골막국수
3076,현대 프리미엄 아울렛 스페이스 원,현대 프리미엄 아울렛 스페이스원
...,...,...
78415,아랑 경양회관 제천점,아랑경양회관 제천점
78515,커피 인터뷰 강경구락부점,커피 인터뷰 강경 구락부점
78663,아름다운 정원과 수목,아름다운 정원과수목
78962,군산 근대 역 사거리,군산근대 역 사거리


time: 16 ms (started: 2024-07-28 21:20:42 +09:00)


In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz, process

def unify_place_names(df, name_column, x_column, y_column, threshold=80, radius=0.001):
    df['UNIFIED_VISIT_AREA_NM'] = df[name_column]  # Create a new column for unified names
    
    coords = df[[x_column, y_column]].dropna()  # Drop rows with missing coordinates
    nn = NearestNeighbors(radius=radius, metric='euclidean')
    nn.fit(coords)
    
    for index, row in df.iterrows():
        if pd.isna(row[x_column]) or pd.isna(row[y_column]):
            continue

        distances, indices = nn.radius_neighbors([[row[x_column], row[y_column]]])
        neighbor_names = df.iloc[indices[0]][name_column].unique()
        
        unified_names = {}
        for name in neighbor_names:
            match = process.extractOne(name, list(unified_names.keys()), scorer=fuzz.ratio)
            if match:
                best_match, score = match
                if score >= threshold:
                    unified_names[name] = unified_names[best_match]
                else:
                    unified_names[name] = name
            else:
                unified_names[name] = name

        df.loc[index, 'UNIFIED_VISIT_AREA_NM'] = unified_names.get(row[name_column], row[name_column])

    return df

# Load the dataset (assuming it's already loaded as df)
# Apply the function to the VISIT_AREA_NM column with consideration of the coordinates
tt = unify_place_names(visit_pre, 'VISIT_AREA_NM', 'X_COORD', 'Y_COORD')

In [18]:
tmp = visit_pre[visit_pre['VISIT_AREA_NM'].str.contains('주차장')][['VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD']]

tmp[tmp['VISIT_AREA_TYPE_CD'] == 1]

,VISIT_AREA_NM,VISIT_AREA_TYPE_CD
2841,선녀바위 공영주차장,1
3047,마장호수 제6주차장,1
10158,화담숲 주차장,1
13989,사나자 계곡 주차장,1
17302,월미도 공영주차장,1
18401,마장호수 제3주차장,1
19647,월미도 공영주차장,1
19759,하늘정원 주차장,1
20104,하늘정원 주차장,1
20552,하늘정원 주차장,1


time: 31 ms (started: 2024-07-28 18:12:43 +09:00)


In [82]:
VIS_codeB

,cd_b,cd_nm
221,21,집
222,22,친구/친지집
223,23,사무실
224,24,숙소
225,13,체험 활동 관광지
226,1,자연관광지
227,2,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)"
228,3,"문화 시설(공연장, 영화관, 전시관 등)"
229,4,"상업지구(거리, 시장, 쇼핑시설)"
230,5,"레저/스포츠 관련 시설(스키, 카트, 수상레저)"


time: 16 ms (started: 2024-07-28 17:45:47 +09:00)


In [80]:
org = pd.read_csv('./visit_pre.csv')

org

,TRAVELER_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM_NP,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,X_COORD,Y_COORD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
0,a000317,a_a000317,6,제주밥상,제주밥상,RES,126.491904,33.482754,3,2,2
1,a000317,a_a000317,7,이중섭미술관,이중섭미술관,TOU,126.564888,33.245904,5,5,5
2,a000317,a_a000317,13,쌍둥이횟집,쌍둥이횟집,RES,126.562928,33.246568,1,1,1
3,a000317,a_a000317,15,천제연폭포,천제연폭포,TOU,126.418375,33.252677,5,5,5
4,a000317,a_a000317,16,돈내코계곡,돈내코계곡,TOU,126.581969,33.300413,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...
25327,d012491,d_d012491,7,사계해변,사계해변,TOU,126.308598,33.228658,4,5,4
25328,d012491,d_d012491,8,일과리방파제 바다(돌고래 출몰지),일과리방파제 바다(돌고래 출몰지),TOU,126.231347,33.245611,3,3,3
25329,d012491,d_d012491,9,노을해안로(돌고래 출몰지),노을해안로(돌고래 출몰지),TOU,126.201377,33.247515,4,4,4
25330,d012491,d_d012491,10,도구리알,도구리알,TOU,126.172075,33.272970,5,5,5


time: 47 ms (started: 2024-07-28 17:43:31 +09:00)


In [13]:
visit_pre = pd.read_csv('visit_pre.csv')

visit_pre

,TRAVELER_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM_NP,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,X_COORD,Y_COORD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
0,a000317,a_a000317,6,제주밥상,제주밥상,RES,126.491904,33.482754,3,2,2
1,a000317,a_a000317,7,이중섭미술관,이중섭미술관,TOU,126.564888,33.245904,5,5,5
2,a000317,a_a000317,13,쌍둥이횟집,쌍둥이횟집,RES,126.562928,33.246568,1,1,1
3,a000317,a_a000317,15,천제연폭포,천제연폭포,TOU,126.418375,33.252677,5,5,5
4,a000317,a_a000317,16,돈내코계곡,돈내코계곡,TOU,126.581969,33.300413,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...
25327,d012491,d_d012491,7,사계해변,사계해변,TOU,126.308598,33.228658,4,5,4
25328,d012491,d_d012491,8,일과리방파제 바다(돌고래 출몰지),일과리방파제 바다(돌고래 출몰지),TOU,126.231347,33.245611,3,3,3
25329,d012491,d_d012491,9,노을해안로(돌고래 출몰지),노을해안로(돌고래 출몰지),TOU,126.201377,33.247515,4,4,4
25330,d012491,d_d012491,10,도구리알,도구리알,TOU,126.172075,33.272970,5,5,5


time: 47 ms (started: 2024-07-28 16:32:06 +09:00)


In [19]:
visit_pre[visit_pre['VISIT_AREA_NM_NP'].str.contains('역')]

,TRAVELER_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM_NP,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,X_COORD,Y_COORD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
1139,d000221,d_d000221,11,역전할머니맥주,역전할머니맥주,RES,126.665619,33.541652,5,4,4
1150,d000221,d_d000221,30,역전할머니맥주 서귀포신시가지점,역전할머니맥주 서귀포신시가지점,RES,126.508065,33.252786,5,4,4
2291,d000451,d_d000451,11,역전할머니맥주 서귀포신시가지점,역전할머니맥주 서귀포신시가지점,RES,126.508065,33.252786,4,4,4
2325,d000458,d_d000458,20,하바나미역,하바나미역,RES,126.229547,33.388134,4,4,4
2905,d000607,d_d000607,13,역전할머니맥주 제주동홍점,역전할머니맥주 제주동홍점,RES,126.573195,33.256744,4,4,3
6068,d001443,d_d001443,22,역전할머니맥주 제주시청점,역전할머니맥주 제주시청점,RES,126.528895,33.500113,4,3,3
6138,d001455,d_d001455,14,역전할머니맥주 연동점,역전할머니맥주 연동점,RES,126.491652,33.485627,3,2,3
7791,d001788,d_d001788,15,역전할머니맥주 서귀포신시가지점,역전할머니맥주 서귀포신시가지점,TOU,126.508065,33.252786,3,3,3
8353,d001916,d_d001916,20,고향역,고향역,RES,126.224065,33.386665,3,3,3
14529,d004093,d_d004093,36,신화월드 테마파크 C-4구역 전기차충전소 전기차충전소,신화월드 테마파크 C-4구역 전기차충전소 전기차충전소,TOU,126.315800,33.302900,3,3,3


time: 15 ms (started: 2024-07-28 16:32:49 +09:00)


In [30]:
visit_pre['VISIT_AREA_NM_NP'].value_counts()

VISIT_AREA_NM_NP
서귀포매일올레시장         528
함덕해수욕장            387
제주동문시장            349
협재해수욕장            329
새별오름              241
                 ... 
금성우동                1
한림국수집               1
귤빛정원                1
더팔당제주               1
노을해안로(돌고래 출몰지)      1
Name: count, Length: 5375, dtype: int64

time: 16 ms (started: 2024-07-28 16:57:54 +09:00)


In [63]:
visit_pre[['VISIT_AREA_NM_NP', 'VISIT_AREA_NM']]

,VISIT_AREA_NM_NP,VISIT_AREA_NM
0,제주밥상,제주밥상
1,이중섭미술관,이중섭미술관
2,쌍둥이횟집,쌍둥이횟집
3,천제연폭포,천제연폭포
4,돈내코계곡,돈내코계곡
...,...,...
25327,사계해변,사계해변
25328,일과리방파제 바다(돌고래 출몰지),일과리방파제 바다(돌고래 출몰지)
25329,노을해안로(돌고래 출몰지),노을해안로(돌고래 출몰지)
25330,도구리알,도구리알


time: 16 ms (started: 2024-07-28 04:11:33 +09:00)


In [66]:
visit_pre[visit_pre['VISIT_AREA_NM_NP'] != visit_pre['VISIT_AREA_NM']]

,TRAVELER_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM_NP,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,X_COORD,Y_COORD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
1840,d000349,d_d000349,9,오설록티뮤지엄,오설록티뮤지엄 주차장,TOU,126.289487,33.305289,3,1,2
8240,d001883,d_d001883,56,연돈볼카츠 제주사수점,연돈볼카츠 제주사수점 주차장,RES,126.478901,33.509559,4,4,4
8692,d002011,d_d002011,20,오설록티뮤지엄,오설록티뮤지엄 주차장,TOU,126.289487,33.305289,5,5,5
22289,d009638,d_d009638,20,해녀김밥 함덕본점,해녀김밥 함덕본점 주차장,RES,126.665297,33.542640,4,4,4


time: 0 ns (started: 2024-07-28 04:11:59 +09:00)


In [73]:
visit_pre[visit_pre['TRAVELER_ID'].isnull()]

,TRAVELER_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM_NP,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,X_COORD,Y_COORD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION
71,NaN,a_a012195,12,고집돌우럭 중문점,고집돌우럭 중문점,RES,126.416705,33.257981,5,5,5
442,NaN,d_d000084,9,소낭집,소낭집,RES,126.516443,33.238158,4,4,4
551,NaN,d_d000105,16,중문그때그집,중문그때그집,RES,126.390090,33.264231,4,4,4
606,NaN,d_d000120,2,제주공항 푸드코트,제주공항 푸드코트,RES,126.493007,33.507008,5,5,5
803,NaN,d_d000159,56,제주돔베고기집,제주돔베고기집,RES,126.478351,33.487781,5,4,3
...,...,...,...,...,...,...,...,...,...,...,...
23809,NaN,d_d010401,8,쌍둥이횟집,쌍둥이횟집,RES,126.562928,33.246568,2,2,1
24473,NaN,d_d010849,20,중문해녀의집,중문해녀의집,RES,126.419139,33.242391,4,5,5
24482,NaN,d_d010849,33,원조마라도짜장면집,원조마라도짜장면집,RES,126.266883,33.118685,5,5,5
24620,NaN,d_d011338,7,제주빵집,제주빵집,RES,126.383179,33.393178,4,3,3


time: 15 ms (started: 2024-07-28 04:13:34 +09:00)


In [77]:
pd.read_csv('./data/bami/total/방문지정보.csv')['VISIT_AREA_NM'].unique()

array(['집', '화성 관광열차 안내소 연무대 매표소', '창룡문', ..., '만천하 스카이워크 짚라인 매표소', '볼프',
       '차반'], dtype=object)

time: 297 ms (started: 2024-07-28 04:15:33 +09:00)


In [37]:
visit_info.columns

Index(['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM',
       'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR',
       'X_COORD', 'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD', 'POI_ID', 'POI_NM',
       'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN',
       'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'SGG_CD', 'region'],
      dtype='object')

time: 0 ns (started: 2024-07-28 03:56:55 +09:00)


In [38]:
org.columns

Index(['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM',
       'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR',
       'X_COORD', 'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD', 'POI_ID', 'POI_NM',
       'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN',
       'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'SGG_CD', '계절'],
      dtype='object')

time: 0 ns (started: 2024-07-28 03:57:00 +09:00)


In [6]:
org = pd.read_csv('traveler_master.csv')

org

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,계절
0,b011774,31,여,30,6,1,1,1,3.0,NaN,4,NaN,2,1,50,50110,11,11620,26,26500,4,2,2,4,3,3,2,6,울산광역시,경남,2인 가족 여행,2022-10-25~2022-10-26,2,1.0,5.0,1,가을
1,d009928,29,여,20,6,2,1,4,12.0,NaN,2,6.0,3,3,50,50110,26,26350,11,11680,6,1,2,3,1,6,1,7,광주광역시,제주,2인 여행(가족 외),2022-10-30~2022-11-01,2,1.0,5.0,1,가을
2,d010131,26,남,20,7,2,1,1,12.0,NaN,3,NaN,4,2,31,31140,31,31110,11,11110,6,2,1,6,5,6,2,7,부산광역시,제주,나홀로 여행,2022-11-02~2022-11-04,5,2.0,10.0,0,가을
3,d006004,50,여,20,6,1,1,1,3.0,NaN,4,NaN,2,2,11,11500,26,26350,50,50130,5,7,6,2,2,5,3,6,제주특별자치도,제주,2인 여행(가족 외),2022-10-30~2022-10-31,3,1.0,7.0,1,가을
4,d002965,50,여,30,6,1,2,4,3.0,NaN,1,4.0,1,1,50,50130,27,27290,47,47110,1,7,1,1,7,7,1,4,제주특별자치도,제주,자녀 동반 여행,2022-10-08~2022-10-10,3,1.0,8.0,3,가을
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,d002221,11,남,30,6,1,1,1,2.0,NaN,4,NaN,3,4,50,50110,26,26710,42,42150,2,1,1,2,3,4,6,6,서울특별시,제주,2인 여행(가족 외),2022-10-02~2022-10-05,2,3.0,5.0,1,가을
3196,d002109,41,여,30,5,1,1,1,3.0,NaN,4,NaN,3,4,50,50130,46,46130,42,42210,2,1,2,3,2,4,2,7,경기도,제주,나홀로 여행,2022-10-04~2022-10-07,2,1.0,4.0,0,가을
3197,d006910,50,여,30,5,1,1,1,4.0,NaN,3,NaN,2,1,50,50130,50,50110,26,26110,4,4,2,2,6,4,6,7,제주특별자치도,제주,나홀로 여행,2022-10-18~2022-10-19,1,4.0,5.0,0,가을
3198,d010634,11,여,20,6,1,1,4,3.0,NaN,3,10.0,3,1,26,26350,50,50130,26,26710,4,1,2,5,3,5,2,7,서울특별시,제주,2인 여행(가족 외),2022-11-05~2022-11-07,2,5.0,1.0,1,가을


time: 31 ms (started: 2024-07-28 03:44:58 +09:00)
